In [344]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [345]:
%cd C:\Users\luzhbin.aleksey\Desktop\analytics

halfYear = pd.read_csv('halfYear.csv', sep = ';',
                    names = ['Dog', 'LoanDate', 'Product', 'NumOfFinishedLoansBefore', 'Status', 'DaysOfOverdue'],
                    skiprows = 1,
                    encoding = 'cp1251')

halfYear['OFF'] = list(map(lambda x: 'OFF' in x, halfYear.Dog))

pstv_povtor_off = halfYear[(halfYear.OFF == True) & (halfYear.Status == 'Заем погашен') & (halfYear.DaysOfOverdue == 0)\
                                        & (halfYear.NumOfFinishedLoansBefore != 0)\
                                        & (halfYear.Product != 'Для сотрудников (ФМ) от 10500 от 29.01.2019')\
                                        & (halfYear.Product != 'Для сотрудников (ФМ) от 15500 от 29.01.2019')\
                                        & (halfYear.Product != 'Для сотрудников (ФМ) от 2000 от 29.01.2019')\
                                        & (halfYear.Product != 'Для сотрудников от 10 500 (ФМ) от 24.06.18')\
                                        & (halfYear.Product != 'Для сотрудников от 15 500 (ФМ) от 24.06.18')\
                                        & (halfYear.Product != 'Для сотрудников от 2 000 (ФМ) от 24.06.18')\
                                        & (halfYear.Product != 'ПСК ДОВЕРИЕ')]
pstv_povtor_off['Default'] = 0

ngtv_povtor_off = halfYear[(halfYear.OFF == True) & (halfYear.Status == 'Заем просрочен') & (halfYear.DaysOfOverdue > 30)\
                                        & (halfYear.NumOfFinishedLoansBefore != 0)
                                        & (halfYear.Product != 'Для сотрудников от 10 500 (ФМ) от 24.06.18')\
                                        & (halfYear.Product != 'Для сотрудников от 15 500 (ФМ) от 24.06.18')]
ngtv_povtor_off['Default'] = 1

def_povtor_off = pstv_povtor_off.append(ngtv_povtor_off, ignore_index = True)
def_povtor_off.drop(['LoanDate', 'Product', 'Status', 'OFF', 'DaysOfOverdue'], axis = 1, inplace = True)

print('positive: {}'.format(def_povtor_off[def_povtor_off.Default == 0].shape[0]))
print('negative: {}'.format(def_povtor_off[def_povtor_off.Default == 1].shape[0]))
print('alltogether: {}'.format(def_povtor_off.shape[0]))

C:\Users\luzhbin.aleksey\Desktop\analytics
positive: 11194
negative: 1069
alltogether: 12263


In [346]:
%cd zvk
path = glob('*.csv')

zvk = pd.read_csv(path[0], sep = ';',
                    skiprows = 1, decimal = ',',
                    dtype = {'LoanSum': 'float'},
                    names = ['Dog', 'DateDog', 'Region', 'ProductType', 'Product', 'Status', 'LoanSum', 'Duration',
                             'Name', 'Gender', 'BD', 'NumOfLoansBefore', 'NumOfzvkBefore', 'NumOfLoansPayed'],
                    usecols = [0, 1, 4, 6, 7, 8, 9, 10, 14, 16, 17, 27, 34, 37], encoding = 'cp1251')

for i in range(1, len(path)):
    tmp = pd.read_csv(path[i], sep = ';',
                    skiprows = 1, decimal = ',',
                    dtype = {'LoanSum': 'float'},
                    names = ['Dog', 'DateDog', 'Region', 'ProductType', 'Product', 'Status', 'LoanSum', 'Duration',
                             'Name', 'Gender', 'BD', 'NumOfLoansBefore', 'NumOfzvkBefore', 'NumOfLoansPayed'],
                    usecols = [0, 1, 4, 6, 7, 8, 9, 10, 14, 16, 17, 27, 34, 37], encoding = 'cp1251')
    zvk = zvk.append(tmp, ignore_index = True)

zvk.drop(['DateDog', 'ProductType', 'Product', 'Status', 'Name', 'NumOfLoansBefore', 'NumOfLoansPayed'],
                                                                                     axis = 1, inplace = True)
print(zvk.shape[0])

C:\Users\luzhbin.aleksey\Desktop\analytics\zvk
710960


In [347]:
merged_def_povtor_off = pd.merge(left = def_povtor_off, right = zvk, left_on = 'Dog', right_on = 'Dog', how = 'left')
merged_def_povtor_off.dropna(inplace = True)
merged_def_povtor_off.shape[0]

12261

In [348]:
merged_def_povtor_off['Sum'] = 0

merged_def_povtor_off.Sum.loc[(merged_def_povtor_off.LoanSum >= 2000)\
                              & (merged_def_povtor_off.LoanSum < 5000)] = '2000-5000'
merged_def_povtor_off.Sum.loc[(merged_def_povtor_off.LoanSum >= 5000)\
                              & (merged_def_povtor_off.LoanSum < 10000)] = '5000-10000'
merged_def_povtor_off.Sum.loc[(merged_def_povtor_off.LoanSum >= 10000)\
                              & (merged_def_povtor_off.LoanSum < 15000)] = '10000-15000'
merged_def_povtor_off.Sum.loc[(merged_def_povtor_off.LoanSum >= 15000)\
                              & (merged_def_povtor_off.LoanSum < 20000)] = '15000-20000'
merged_def_povtor_off.Sum.loc[(merged_def_povtor_off.LoanSum >= 20000)\
                              & (merged_def_povtor_off.LoanSum <= 30000)] = '20000-3000'

In [349]:
merged_def_povtor_off['Age'] = list(map(lambda x: (dt.today() - dt.strptime(x, "%Y-%m-%d")).days // 365,
                                        merged_def_povtor_off.BD.values))
merged_def_povtor_off['Duration'] = list(map(int, merged_def_povtor_off.Duration.values))
merged_def_povtor_off.drop(['BD', 'LoanSum'], axis = 1, inplace = True)

In [350]:
merged_def_povtor_off = merged_def_povtor_off.join(pd.get_dummies(merged_def_povtor_off[['Gender', 'Sum', 'Region']]),
                                                   how ='left')
print(merged_def_povtor_off.shape)

(12261, 96)


In [351]:
merged_def_povtor_off['Def'] = merged_def_povtor_off.Default
merged_def_povtor_off.drop(['Dog', 'Default', 'Gender', 'Sum', 'Region'], axis = 1, inplace = True)
merged_def_povtor_off[:5]

,NumOfFinishedLoansBefore,Duration,NumOfzvkBefore,Age,Gender_Женский,Gender_Мужской,Sum_10000-15000,Sum_15000-20000,Sum_2000-5000,Sum_20000-3000,...,Region_Хакасия,Region_Ханты-Мансийский,Region_Ханты-Мансийский Автономный округ - Югра,Region_Челябинская,Region_Читинская,Region_Чувашская,Region_Чувашская Республика -,Region_Ямало-Ненецкий,Region_Ярославская,Def
0,7,16,7.0,37,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,19,29,19.0,69,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,31,11.0,53,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,29,2.0,60,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,16,4.0,41,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [352]:
X = merged_def_povtor_off.iloc[:, :-1].values
y = merged_def_povtor_off.iloc[:, -1].values

In [353]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 42)

In [354]:
clf = LogisticRegression(penalty='l1').fit(X_train, y_train)

In [356]:
print('Значение метрики качества модели: {:.2f}'.format(roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])))

Значение метрики качества модели: 0.77
